Proste funkcje można w miarę bezproblemowo zróżniczkować symbolicznie

In [3]:
function f(x::Number, y::Number)
    x^2 + y^2
end

f (generic function with 1 method)

In [5]:
import Pkg
Pkg.add("SymEngine")
using SymEngine

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [11]:
x, y = symbols("x y")

(x, y)

In [14]:
∂f∂x = diff(f(x, y), x);
∂f∂y = diff(f(x, y), y);
∇f = Jf = [∂f∂x ∂f∂y]

1×2 Matrix{Basic}:
 2*x  2*y

W efekcie otrzymujemy funkcje, które możemy zewaluować podstawiająć konkretne wartości za symbole

In [15]:
subs.(∇f, x => 3, y => -2)

1×2 Matrix{Basic}:
 6  -4

In [16]:
∂²f∂x² = diff(∂f∂x, x)
∂²f∂xy = diff(∂f∂x, y)
∂²f∂yx = diff(∂f∂y, x)
∂²f∂y² = diff(∂f∂y, y)
Hf = [∂²f∂x² ∂²f∂xy;
      ∂²f∂yx ∂²f∂y²]

2×2 Matrix{Basic}:
 2  0
 0  2

Natomiast w ten sposób nie zróżniczkujemy dowolnego kodu:

In [17]:
function g(x, y)
    r = 1.0
    for i=1:y
        r *= x
    end
    return r
end
dgdx = diff(g(x, y), x)

LoadError: MethodError: no method matching (::Colon)(::Int64, ::Basic)
[0mClosest candidates are:
[0m  (::Colon)(::T, ::Any, [91m::T[39m) where T<:Real at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/range.jl:41
[0m  (::Colon)(::A, ::Any, [91m::C[39m) where {A<:Real, C<:Real} at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/range.jl:10
[0m  (::Colon)(::T, ::Any, [91m::T[39m) where T at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/range.jl:40
[0m  ...

In [22]:
2.0^3

8.0

A jeśli już coś zróżniczkujemy to może się okazać, że wynikowa "formuła" jest skomplikowana:

In [23]:
function Babylonian(x; N = 10)
    t = (1+x)/2
    for i = 2:N; t=(t + x/t)/2  end
    t
end

Babylonian (generic function with 1 method)

In [24]:
diff( Babylonian(x; N=3), x ) |> expand |> display

1/8 + (-1/2)*x/(1 + 2*x + x^2) + (-1/2)*x/(1/4 + (1/2)*x + 2*x/(1 + x) + 4*x^2/(1 + x)^2 + 2*x^2/(1 + x) + (1/4)*x^2) - 2*x/((1 + x)*(1/4 + (1/2)*x + 2*x/(1 + x) + 4*x^2/(1 + x)^2 + 2*x^2/(1 + x) + (1/4)*x^2)) + 2*x^2/((1 + 2*x + x^2)*(1/4 + (1/2)*x + 2*x/(1 + x) + 4*x^2/(1 + x)^2 + 2*x^2/(1 + x) + (1/4)*x^2)) + (1/2)*(1 + x)^(-1) + (1/2 + (1/2)*x + 2*x/(1 + x))^(-1)

Konkluzja: czasami można sobie pozwolić na symboliczne różniczkowanie, jednak bardzo częstu nie jesteśmy zainteresowani **wyrażeniem** matematycznym na (przykładowo) gradient, a bardziej interesuje nas **wartość** tego gradientu.

In [25]:
subs(diff( Babylonian(x; N=4), x ), x=>9)

159131117/954408050

In [26]:
.5/sqrt(9)

0.16666666666666666

In [28]:
1196926150789296566693143581061850072609040839619092001190012793612024264402514477525822274500848646803285850404514741462745168857425589670188450088386973448912254717307444299694451768532950078029483616236446946973214526616393324302290299790313932239651172068593937223810650958945965400318798237004590014288816296255722965401774305633028102539947312592624615245558015196948459860368812580962610361484524164635794019377811927952648127261157243631608050039832057121314022592048303910893532089231145625590726442851230576254460942420236147577116520850262965689572221723961399396340899564448510977497061599107921032259053/7181556904735779400158861486371100435654245037714552007140076761672145586415086865154933647005091880819715102427088448776471013144553538021130700530321840693473528303844665798166710611197700468176901697418681681839287159698359945813741798741883593437907032411563623342863905753675792401912789422027540085671456621058376862053142025277423725455140573475452857943006876777114569075799435197307742586270969343263045194657493126809395727025482847257648445055989853107215908866191353164260567258338764252273504972594774819199415487978592905083392821987846173969159311358680901557370094219272887270129752253544846921768050

0.1666666666666666666666666666666666666666666666666666666666666666666666666666652